# Data Wrangling in Pandas - Worked Examples


# Reviewing our earlier application of Data Wrangling to Craigslist Data

In [ ]:
# import libraries and read in the csv file
import re as re, pandas as pd, numpy as np, requests, json
df = pd.read_csv('bay.csv')
print(df[:5])

# clean price and neighborhood
df.price = df.price.str.strip('$').astype('float64')
df.neighborhood = df.neighborhood.str.strip().str.strip('(').str.strip(')')

# break out the date into month day year columns
df['month'] = df['date'].str.split().str[0]
df['day'] = df['date'].str.split().str[1].astype('int32')
df['year'] = df['date'].str.split().str[2].astype('int32')
del df['date']

def clean_br(value):
    if isinstance(value, str):
        end = value.find('br')
        if end == -1:
            return None
        else:
            start = value.find('/') + 2
            return int(value[start:end])
df['bedrooms'] = df['bedrooms'].map(clean_br)

def clean_sqft(value):
    if isinstance(value, str):
        end = value.find('ft')
        if end == -1:
            return None
        else:
            if value.find('br') == -1:
                start = value.find('/') + 2
            else:
                start = value.find('-') + 2
            return int(value[start:end])
df['sqft'] = df['sqft'].map(clean_sqft)



df.head()

## Let's do some wrangling on this dataset:
1. Find outliers in rent, say below 200 or above 10,000
1. Analyze the data without missing data
1. Create a dataset that removes the outliers

In [ ]:
df['price'].dropna().describe()

In [ ]:
df['price'][(df['price'] < 200)].dropna().describe()

In [ ]:
df['price'][(df['price'] > 10000)].dropna().describe()

In [ ]:
# Let's get a quantile value at the 99 percentile to see the value that the top one percent of our records exceed
df['price'].dropna().quantile(.99)

In [ ]:
filtered = df[(df['price'] < 10000) & (df['price'] > 200)]
filtered.dropna().describe()

## OK, now on your own: 
1. Filter out records with more than 4 bedrooms
2. Create dummy variables for each bedroom count (e.g. bed_1 would have 1 for rows with 1 bedroom, 0 for others), and merge them with the dataframe
3. Filter sqft < 500 and > 3000
4. Create a set of 5 bins for price that include all values, and do counts of how many records are in each category

In [ ]:
BedFilter = filtered[(filtered['bedrooms']<5)]
BedFilter.dropna().describe()

In [ ]:
dummies = pd.get_dummies(BedFilter['bedrooms'],prefix='bedrooms')
BedFilterWithDummies = BedFilter.join(dummies)
BedFilterWithDummies.head()

In [ ]:
bedfilter_dummies2=pd.merge(BedFilter, dummies, left_index=True, right_index=True)
bedfilter_dummies2.head()

In [ ]:
bins = [0, 500, 1000, 1500, 2000, 99999]
cats = pd.cut(df['price'], bins)
pd.value_counts(cats)